In [11]:
import pandas as pd
import yfinance as yf

#dfSPY=yf.download("^GSPC",start='2011-01-05', end='2021-01-05')
dfSPY=yf.download("^RUI",start='2011-01-05', end='2021-01-05')
#dfSPY=yf.download("AMZN",start='2011-01-05', end='2021-01-05')

dfSPY=dfSPY[dfSPY.High!=dfSPY.Low]
dfSPY.reset_index(inplace=True)
dfSPY.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-01-05,704.739990,707.419983,699.049988,706.869995,706.869995,0
1,2011-01-06,702.380005,707.929993,700.679993,705.479980,705.479980,0
2,2011-01-07,706.830017,707.929993,698.780029,704.320007,704.320007,0
3,2011-01-10,703.229980,704.770020,699.020020,703.760010,703.760010,0
4,2011-01-11,705.090027,707.929993,703.859985,706.520020,706.520020,0


In [12]:
import pandas_ta as ta
dfSPY['EMA']=ta.ema(dfSPY.Close, length=200)#sma ema
dfSPY['RSI']=ta.rsi(dfSPY.Close, length=2)
a=ta.adx(dfSPY.High, dfSPY.Low, dfSPY.Close, length=14)
dfSPY=dfSPY.join(a.ADX_14)
#dfSPY.ta.indicators()
#help(ta.adx)
dfSPY.dropna(inplace=True)
dfSPY.reset_index(inplace=True)
dfSPY[420:425]

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,RSI,ADX_14
420,619,2013-06-24,879.289978,881.609985,863.799988,871.000000,871.000000,0,843.446264,8.366059,22.398568
421,620,2013-06-25,870.909973,882.559998,870.909973,879.539978,879.539978,0,843.805406,50.977635,23.995480
422,621,2013-06-26,879.820007,890.020020,879.820007,887.969971,887.969971,0,844.244854,74.441711,24.583740
423,622,2013-06-27,890.409973,897.500000,890.409973,894.150024,894.150024,0,844.741423,84.981484,24.322974
424,623,2013-06-28,892.989990,895.809998,887.580017,890.669983,890.669983,0,845.198424,58.030359,24.270829


In [15]:
def addemasignal(df, backcandles):
    emasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.EMA[i]:
                dnt=0
            if df.Low[i]<=df.EMA[i]:
                upt=0
        if upt==1 and dnt==1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row]=3
        elif upt==1:
            emasignal[row]=2
        elif dnt==1:
            emasignal[row]=1
    df['EMASignal'] = emasignal

addemasignal(dfSPY,6)

In [36]:
def totalSignal(df, ADX):
    ordersignal=[0]*len(df)
    for i in range(0, len(df)):
        if  df.RSI[i]<=25 and ADX[i]>=20 and df.EMASignal[i]==2:
            ordersignal[i]=2
    df['ordersignal']=ordersignal

totalSignal(dfSPY, dfSPY.ADX_14)

In [37]:
dfSPY[dfSPY.ordersignal!=0]

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,RSI,ADX_14,EMASignal,ordersignal,pointpos
69,268,2012-01-30,728.239990,728.239990,719.809998,726.719971,726.719971,0,701.354194,21.705780,30.994876,2,2,719.807998
70,269,2012-01-31,728.869995,729.770020,723.219971,726.330017,726.330017,0,701.602709,18.371331,31.242481,2,2,723.217971
78,277,2012-02-10,748.820007,748.820007,741.090027,743.969971,743.969971,0,704.816182,23.115542,39.754236,2,2,741.088027
81,280,2012-02-15,749.169983,751.719971,743.440002,744.770020,744.770020,0,706.076269,24.211385,39.951479,2,2,743.438002
93,292,2012-03-05,758.270020,758.270020,752.479980,755.349976,755.349976,0,711.782299,22.146687,34.811322,2,2,752.477980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,2471,2020-10-30,1847.040039,1847.040039,1807.069946,1825.670044,1825.670044,0,1774.034730,19.759005,21.040202,2,2,1807.067946
2299,2498,2020-12-09,2087.350098,2092.840088,2059.889893,2067.830078,2067.830078,0,1830.439805,24.468482,28.986735,2,2,2059.887893
2301,2500,2020-12-11,2068.879883,2068.879883,2048.719971,2066.100098,2066.100098,0,1835.138050,22.355420,28.050670,2,2,2048.717971
2302,2501,2020-12-14,2066.520020,2085.750000,2058.929932,2059.219971,2059.219971,0,1837.367721,9.724099,28.132860,2,2,2058.927932


# Visualization

In [38]:
import numpy as np
def pointpos(x):
    if x['ordersignal']==1:
        return x['High']+2e-3
    elif x['ordersignal']==2:
        return x['Low']-2e-3
    else:
        return np.nan

dfSPY['pointpos'] = dfSPY.apply(lambda row: pointpos(row), axis=1)

In [39]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = dfSPY[1000:3200].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=6, color="MediumPurple"),
                name="Signal")
#fig.update(layout_yaxis_range = [300,420])
fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(autosize=False, width=600, height=600,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")
fig.show()

In [40]:
dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [47]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize
    
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        perc=0.02
        if len(self.trades)>0:
            if self.data.index[-1]-self.trades[-1].entry_time>=1000:
                self.trades[-1].close()
            if self.trades[-1].is_long and self.data.RSI[-1]>=75:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=25:
                self.trades[-1].close()
        
        if self.signal==2 and len(self.trades)==0:  
            sl1 = min(self.data.Low[-1],self.data.Low[-2])*(1-perc)
            #tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            #self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.buy(sl=sl1,size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=1000, margin=1/5, commission=.000)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    2315.0
Duration                               2315.0
Exposure Time [%]                   19.343696
Equity Final [$]                  3018.098156
Equity Peak [$]                   3595.734056
Return [%]                         201.809816
Buy & Hold Return [%]              213.320124
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -35.778367
Avg. Drawdown [%]                   -7.124936
Max. Drawdown Duration                  421.0
Avg. Drawdown Duration              57.212121
# Trades                                102.0
Win Rate [%]                        73.529412
Best Trade [%]                       4.431525
Worst Trade [%]                     -3.315564
Avg. Trade [%]                    

In [49]:
bt.plot()

Row(id='7735', ...)